# Kaggle
## Competição DSA de Machine Learning - Dezembro 2019

Versão 1.0.0: LB = 0.50744
- modelo: NN com 3 camadas
- features categoricas: removido
- dados missing: atribuído o valor medio
- feature selection: 25

## Importing libraries

In [45]:
# Importar os principais pacotes
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm_notebook as tqdm
import re
import random as rd
import os
import codecs
import time
import datetime
import gc
from numba import jit
from collections import Counter
import copy
from typing import Any

seed = 12345
np.random.seed(seed)
rd.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

# Evitar que aparece os warnings
import warnings
warnings.filterwarnings("ignore")

# Seta algumas opções no Jupyter para exibição dos datasets
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Variavel para controlar o treinamento no Kaggle
TRAIN_OFFLINE = True

In [51]:
# Importa os pacotes de algoritmos
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb 

# Importa os pacotes de algoritmos de redes neurais (Keras)
import keras
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.utils import to_categorical
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda,BatchNormalization
from keras.layers import Activation
from keras.models import Sequential, Model
from keras.callbacks import Callback,EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
import keras.backend as K
from keras.optimizers import Adam
#from keras_radam import RAdam
from keras import optimizers
from keras.utils import np_utils

# Importa pacotes do sklearn
from sklearn import preprocessing
import sklearn.metrics as mtr
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, log_loss, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn import model_selection
from sklearn.utils import class_weight

from sklearn.preprocessing import StandardScaler as SS
from sklearn.model_selection import train_test_split as TTS

from keras.callbacks import ReduceLROnPlateau as RLRP
from keras.callbacks import EarlyStopping as ES

In [11]:
def read_data():
    
    if TRAIN_OFFLINE:
        print('Carregando arquivo dataset_treino.csv....')
        train = pd.read_csv('../dataset/dataset_treino.csv')
        print('dataset_treino.csv tem {} linhas and {} colunas'.format(train.shape[0], train.shape[1]))

        print('Carregando arquivo dataset_treino_modificado.csv....')
        train_m = pd.read_csv('../dataset/dataset_treino_modificado.csv')
        print('dataset_treino_modificado.csv tem {} linhas and {} colunas'.format(train_m.shape[0], train_m.shape[1]))
        
        print('Carregando arquivo dataset_teste.csv....')
        test = pd.read_csv('../dataset/dataset_teste.csv')
        print('dataset_teste.csv tem {} linhas and {} colunas'.format(test.shape[0], test.shape[1]))

        print('Carregando arquivo dataset_teste_modificado.csv....')
        test_m = pd.read_csv('../dataset/dataset_teste_modificado.csv')
        print('dataset_teste_modificado.csv tem {} linhas and {} colunas'.format(test_m.shape[0], test_m.shape[1]))
        
    else:
        print('Carregando arquivo dataset_treino.csv....')
        train = pd.read_csv('/kaggle/input/competicao-dsa-machine-learning-dec-2019/dataset_treino.csv')
        print('dataset_treino.csv tem {} linhas and {} colunas'.format(train.shape[0], train.shape[1]))
        
        print('Carregando arquivo dataset_treino.csv....')
        test = pd.read_csv('/kaggle/input/competicao-dsa-machine-learning-dec-2019/dataset_teste.csv')
        print('dataset_teste.csv tem {} linhas and {} colunas'.format(test.shape[0], test.shape[1]))
    
    return train, train_m, test, test_m

In [12]:
# Leitura dos dados
train, train_m, test, test_m = read_data()

Carregando arquivo dataset_treino.csv....
dataset_treino.csv tem 114321 linhas and 133 colunas
Carregando arquivo dataset_treino_modificado.csv....
dataset_treino_modificado.csv tem 114321 linhas and 44 colunas
Carregando arquivo dataset_teste.csv....
dataset_teste.csv tem 114393 linhas and 132 colunas
Carregando arquivo dataset_teste_modificado.csv....
dataset_teste_modificado.csv tem 114393 linhas and 51 colunas


In [13]:
train_m.head()

,v31,v129,v50,v110,v66,v47,v38,v113,v56,v79,v24,v71,v74,v101,v3,v62,v30,v85,v72,v67,v100,v94,v84,v111,v95,v106,v108,v22,v125,v112,v52,v91,v107,v10_bin,v14_bin,v26_bin,v28_bin,v34_bin,v46_bin,v55_bin,v57_bin,v58_bin,v65_bin,target
0,0,0,0.111270,0,0,0,0,0,0,0,0,0,0,0.386152,0,1,0,0.269716,1,0.598997,0.141689,0.281723,0.139374,0.338176,0.212921,0.495177,0.131094,0,0,0,0,0,0,49,45,122,175,141,251,138,9,106,128,0
1,0,0,0.163392,0,1,1,0,1,1,1,1,1,0,0.000000,0,1,1,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,0,1,0,0,61,35,0,0,175,0,0,0,0,0,1
2,0,0,0.146414,0,0,2,0,0,2,2,2,1,0,0.233377,0,1,-1,0.328590,1,0.441279,0.764561,0.239724,0.172676,0.340090,0.163506,0.407940,0.135128,2,2,1,0,0,0,61,58,30,175,62,124,138,21,54,37,1
3,0,0,0.053418,0,2,1,0,0,-1,3,1,1,0,0.000000,0,2,-1,0.000000,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3,3,2,2,1,1,38,43,0,0,62,0,0,0,0,0,0
4,1,0,0.171427,1,1,3,6,-1,3,4,0,1,1,0.398321,0,0,-1,0.114583,6,0.506923,0.973248,0.197568,0.128673,0.236744,0.170047,0.612259,0.132526,4,4,3,3,0,0,55,137,54,69,120,15,32,61,22,151,1


## Criar e avaliar alguns algoritmos de Machine Learning

### One-Hot encoding / Scaling / Feature Selection

In [63]:
# Criar um dataset somente com as colunas mais importantes conforme Feature Selection
train_x = train_m.drop(['target'], axis=1)
train_y = train_m['target']
train_y = to_categorical(train_y)

In [64]:
train_x.shape, train_y.shape

((114321, 43), (114321, 2))

### Criação e Validação dos Modelos de Machine Learning

### Modelo Rede Neural (MLP)

In [16]:
gc.collect()

87

In [65]:
def create_model(size):
    model = Sequential()
    
    model.add(Dense(256, input_dim=size, activation='hard_sigmoid'))
    model.add(Dropout(0.625))

    model.add(Dense(256, activation='hard_sigmoid'))
    model.add(Dropout(0.625))
    
    #model.add(Dense(32, activation='hard_sigmoid'))
    #model.add(Dropout(0.1))
    
    model.add(Dense(16, activation='hard_sigmoid'))
    model.add(Dropout(.25))
    
    #model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(2, activation='softmax'))
    
    optimizer = Adam(lr=0.007, epsilon=3e-7, decay=3e-7)
    
    model.compile(optimizer=optimizer,
                 loss='categorical_crossentropy',
                 metrics=['categorical_accuracy'])
    return model

In [55]:
def get_nn(x_tr,y_tr,x_val,y_val,shape):
    K.clear_session()
    
    inp = Input(shape = (x_tr.shape[1],))

    x = Dense(512, input_dim=x_tr.shape[1], activation='relu')(inp)
    x = Dropout(0.5)(x)    
    x = BatchNormalization()(x)
    
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)    
    x = BatchNormalization()(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    
    out = Dense(4, activation='softmax')(x)
    model = Model(inp,out)
    
    model.compile(optimizer = 'Adam', #'RMSprop', #optimizers.adam(learning_rate=0.1), #, beta_1=0.9, beta_2=0.99),
                  loss='categorical_crossentropy', 
                  metrics=['categorical_accuracy'])
     
    es = EarlyStopping(monitor='val_loss', 
                       mode='min',
                       restore_best_weights=True, 
                       verbose=1, 
                       patience=20)

    mc = ModelCheckpoint('best_model.h5',
                         monitor='val_loss',
                         mode='min',
                         save_best_only=True, 
                         verbose=1, 
                         save_weights_only=True)

    rl = ReduceLROnPlateau(monitor='val_loss', 
                           factor=0.1, 
                           patience=10, 
                           verbose=1, 
                           epsilon=1e-4, 
                           mode='min')

    model.fit(x_tr, y_tr,
              validation_data=[x_val, y_val],
              callbacks=[es,mc,rl],
              epochs=250, 
              batch_size=64,
              verbose=1,
              #class_weight=class_weight_y,
              shuffle=True)
    
    model.load_weights("best_model.h5")
    
    y_pred = model.predict(x_val)
    y_valid = y_val
             
    logloss = log_loss(y_valid, y_pred, eps=1e-15)

    return model, logloss

In [ ]:
batch_size = 128
epochs = 100

model.fit(train_x, 
          train_y, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1, 
          callbacks=[earlyStopping, mcp_save, reduce_lr_loss], 
          validation_split=0.1)

In [46]:
def graph(history):
    # summarize history for accuracy
    import matplotlib.pyplot as plt
    print(f'Accuracy:')
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model loss')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='lower right')
    #plt.ylim(top=.5, bottom=-.5)
    plt.savefig('history_acc.png')
    plt.show()
    
    #summarize history for loss
    print('Loss:')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    #plt.ylim(top=.5, bottom=-.5)
    plt.savefig('history_loss.png')
    plt.show()

In [47]:
def run_model(comp_train_df, comp_test_df):
    print(f'Run model')
    scale = SS(with_mean=False)
    #x_col = [col for col in comp_train_df.columns if col not in ['installation_id', 'accuracy_group']]
    #y = to_categorical(comp_train_df['accuracy_group'], num_classes=4)
    
    x_train, x_val, y_train, y_val = TTS(comp_train_df, comp_test_df, random_state=seed)
    
    model = create_model(x_train.shape[1])
    
    lrr = RLRP(monitor='val_acc',
               patience=5,
               verbose=1,
               factor=0.95,
               min_lr=1e-5)
    es = ES(monitor='val_loss',
           patience=99,
           mode='min',
           verbose=1)
               
    print(f'Model Fitting...')
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
              epochs=1000, batch_size=64, verbose=2,
             callbacks=[lrr, es])
    graph(history)
    
    y_pred = model.predict(x_val)
    y_valid = y_val
             
    logloss = log_loss(y_valid.argmax(axis=1), y_pred.argmax(axis=1), eps=1e-15)
    print(f'Log Loss score: {np.round(logloss,4)}')
    
    return y_pred

In [66]:
y_pred = run_model(train_x, train_y)

Run model
Model Fitting...
Train on 85740 samples, validate on 28581 samples
Epoch 1/1000
 - 3s - loss: 0.5534 - categorical_accuracy: 0.7606 - val_loss: 0.5488 - val_categorical_accuracy: 0.7623
Epoch 2/1000
 - 3s - loss: 0.5510 - categorical_accuracy: 0.7608 - val_loss: 0.5485 - val_categorical_accuracy: 0.7623
Epoch 3/1000
 - 3s - loss: 0.5511 - categorical_accuracy: 0.7608 - val_loss: 0.5485 - val_categorical_accuracy: 0.7623
Epoch 4/1000
 - 3s - loss: 0.5513 - categorical_accuracy: 0.7608 - val_loss: 0.5487 - val_categorical_accuracy: 0.7623
Epoch 5/1000
 - 3s - loss: 0.5510 - categorical_accuracy: 0.7608 - val_loss: 0.5485 - val_categorical_accuracy: 0.7623
Epoch 6/1000
 - 3s - loss: 0.5510 - categorical_accuracy: 0.7608 - val_loss: 0.5497 - val_categorical_accuracy: 0.7623
Epoch 7/1000
 - 3s - loss: 0.5512 - categorical_accuracy: 0.7608 - val_loss: 0.5489 - val_categorical_accuracy: 0.7623
Epoch 8/1000
 - 3s - loss: 0.5510 - categorical_accuracy: 0.7608 - val_loss: 0.5508 - val_

Epoch 69/1000
 - 3s - loss: 0.5512 - categorical_accuracy: 0.7608 - val_loss: 0.5487 - val_categorical_accuracy: 0.7623
Epoch 70/1000
 - 3s - loss: 0.5511 - categorical_accuracy: 0.7608 - val_loss: 0.5513 - val_categorical_accuracy: 0.7623
Epoch 71/1000
 - 3s - loss: 0.5510 - categorical_accuracy: 0.7608 - val_loss: 0.5485 - val_categorical_accuracy: 0.7623
Epoch 72/1000
 - 3s - loss: 0.5511 - categorical_accuracy: 0.7608 - val_loss: 0.5490 - val_categorical_accuracy: 0.7623
Epoch 73/1000
 - 3s - loss: 0.5512 - categorical_accuracy: 0.7608 - val_loss: 0.5485 - val_categorical_accuracy: 0.7623
Epoch 74/1000
 - 3s - loss: 0.5510 - categorical_accuracy: 0.7608 - val_loss: 0.5490 - val_categorical_accuracy: 0.7623
Epoch 75/1000
 - 3s - loss: 0.5511 - categorical_accuracy: 0.7608 - val_loss: 0.5489 - val_categorical_accuracy: 0.7623
Epoch 76/1000
 - 3s - loss: 0.5511 - categorical_accuracy: 0.7608 - val_loss: 0.5505 - val_categorical_accuracy: 0.7623
Epoch 77/1000
 - 3s - loss: 0.5510 - cat

KeyboardInterrupt: 

In [ ]:
%%time

loop = 2
fold = 5

#oof_nn = np.zeros([loop, train_y.shape[0], train_y.shape[1]])
models_nn = []
loglos_csv_nn = []

#class_weight_y = class_weight.compute_class_weight('balanced',np.unique(train_y), train_y)

for k in range(loop):
    kfold = KFold(fold, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_)):
        print("-----------")
        print(f'Loop {k+1}/{loop}' + f' Fold {k_fold+1}/{fold}')
        print("-----------")
        
        tr_x, tr_y = train_x[tr_inds], train_y[tr_inds]
        #val_x, val_y = train_x[val_inds], train_y[val_inds]
        
        # Train NN
        #nn, logloss_nn = get_nn(tr_x, tr_y, val_x, val_y, shape=val_x.shape[0])
        #models_nn.append(nn)
        #print("the %d fold Log-Loss (NN) is %f"%((k_fold+1), logloss_nn))
        #loglos_csv_nn.append(logloss_nn)
        
        #Predict OOF
        #oof_nn[k, val_inds, :] = nn.predict(val_x)
        
    print("PARTIAL: mean Log-Loss (NN) is %f"%np.mean(loglos_csv_nn))        

## Resultado

In [ ]:
model.load_weights('best_model.hdf5')

score = model.evaluate(train_x, train_y, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

In [ ]:
plt.figure(figsize=(18, 8))
plt.subplot(2, 1, 1)
plt.plot(model.history.history["loss"], "o-", alpha=.9, label="loss")
plt.plot(model.history.history["val_loss"], "o-", alpha=.9, label="val_loss")
plt.axhline(1, linestyle="--", c="C2")
plt.legend()
plt.subplot(2, 1, 2)
plt.plot(model.history.history["accuracy"], "o-", alpha=.9, label="accuracy")
plt.plot(model.history.history["val_accuracy"], "o-", alpha=.9, label="val_accuracy")
plt.axhline(.7, linestyle="--", c="C2")
plt.legend()
plt.show()

## Submissions

In [ ]:
test.fillna(test.mean(),inplace=True)
new_test = test.loc[:,best_features['Feature']]
text_x = scaler.fit_transform(new_test)

### Submissions NN

In [ ]:
pred_test = model.predict(text_x)

submission = pd.DataFrame({'ID': test["ID"], 'PredictedProb': pred_test.reshape((pred_test.shape[0]))})
print(submission.head(10))

In [ ]:
submission.to_csv('../submission/submission_nn_v1.csv', index=False)

In [ ]:
submission['PredictedProb'].value_counts(normalize=True)

In [ ]:
plt.hist(submission.PredictedProb)
plt.show()